# CS155 Project 3: Shakespearean Sonnets

In [16]:
import re
import os
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()

## Preprocessing:

Initial Attempt:
- Process the words line by line
- Remove line containing numbering for poem (1, 2, 3, etc.)
- Change all the words to lowercase
- Uses TweetTokenizer to separate words (retains apostrophes and hyphens)
- Remove all punctualization

In [70]:
def preprocess(text):
    # Convert text to dataset.
    lines = text.split('\n')

    obs_counter = 0
    obs = []
    obs_map = {}

    for line in lines:
        # Separate into words using TweetTokenizer and lowercase
        sentence = tknzr.tokenize(line)
        # Skip if line is poem numbering
        if sentence == [] or not sentence[0].isnumeric():
            obs_elem = []
            punct = ".,':;"; 
            
            for word in sentence:
                # Remove intermediate punctuation
                if not word in punct:
                    # Turn to lowercase
                    word = word.lower()
                    if word not in obs_map:
                        # Add unique words to the observations map.
                        obs_map[word] = obs_counter
                        obs_counter += 1

                    # Add the encoded word.
                    obs_elem.append(obs_map[word])

            # Add the encoded sequence.
            obs.append(obs_elem)

    return obs, obs_map

In [71]:
text = open(os.path.join(os.getcwd(), 'data/shakespeare.txt')).read()

In [72]:
obs, obs_map = preprocess(text)